# RankNetを用いた競馬着順予測設計


## 概要

RankNetを用いて競馬の着順予測を行った。

結果、単勝回収率93%-95%の成果を出すことができた(ほぼ同じ条件のlgbmでは70%)。

その上で、いくつかの課題を見出すこともできた。
- 勝率の計算の仕方の最適解がわからない。
- どんな買い方をするのが最適かわからない。
- どの程度のデータ量で検証すれば精度算出できるのかわからない。
etc..


## 目的

競馬の着順予測のため、RankNetの考え方を転用して深層学習を行う。

最終的には以下の2つのうち片方を目標とする。

- 単勝回収率100%以上
- 複勝回収率100%以上

購買は、以下の方法で制御する。

- 該当レースにおいて1位予測された馬の単勝を買う。
- 該当レースにおいて1, 2, 3位予測された馬の複勝を買う。

## 背景

ランキング学習を選択した背景としては、これまでの学習方法ではレース単位での相対評価が難しかったからである。

ランキング学習の手法は多々あるが、今回はその中でRankNetを選択した。理由は後述する。

## アルゴリズム

### ランキング学習

ランキング学習は、その名の通りランキングを予測するための学習方法である。

深層学習におけるその損失関数は、以下に大別される。
- Pointwise
  - 1つのサンプルから損失を定義する。
  - 予測結果とgtラベルとの二乗誤差を最小化する。
  - 要するにただの回帰による多クラス分類問題である。
- Pairwise
  - 2つのサンプルから損失を定義する。
  - サンプルから2つ選択し、確率の差を取る。真の確率はその差が`>0`, `=0`, `<0` それぞれに`1`, `0`, `1/2`となる。
  - 要するに2つのサンプルを比較してどちらが上かを考える。
- Listwise
  - リスト(今回の場合は1レースに出走する馬のリスト)の全体としていい並び順になっているかどうかを指標とする。

Pointwiseでは今回の目的に反し、Listwiseは学習がうまくいかなさそう(競馬は一つのリストが10以上となり、うまくいかなさそう)なので、Pairwiseを選択した。


## RankNet

RankNetによりPairwiseによるランキング学習を行うことができる。
行うことができる、といってもそのようなフレームワークやライブラリがあるわけでもなく、論文の考え方に基づいて普通にコードを書いて実装する。

## 学習設計

### 概要

jrdbからスクレイピングによって取得したデータを加工した上で、
Tensorflowを用いてニューラルネットワークを構築する。

レース単位での学習を実現するためにジェネレーターを用いる。

### データ加工

データの加工は[別ノート](https://colab.research.google.com/drive/1ejO3POljB5LKak7hLqle3XaKnZ7KtwNe?hl=ja)にて記載。

### 特徴量

SEDのデータをベースに特徴量を生成する。
- 馬場データ、天候データ、競技場データなどはOne hot Encodingによりダミー変数化を行う。
- タイム指標として、各レースのレース時間、前3F, 後3FそれぞれについてTSPを算出する。
  - TSPについては、Tomoyuki Takita氏のノートブックを参照。
- 前レースの結果を特徴量に入れるため、過去3戦のTSPを当該レースに加える。

具体的な特徴量は以下。
- 詳細は今度載せます。

In [ ]:
import pandas as pd

sed_df = pd.read_csv('/content/drive/MyDrive/競馬/nakao_work/sed_union_nbt_ounull_getdummies.csv')

sed_df['レースid'] = (sed_df['競争成績キー_年月日'])*10000 + sed_df['レースキー_場コード_1']*1 + sed_df['レースキー_場コード_2']*2 + sed_df['レースキー_場コード_3']*3 + sed_df[ 'レースキー_場コード_4']*4 + sed_df[ 'レースキー_場コード_5']*5 + sed_df[ 'レースキー_場コード_6']*6 + sed_df['レースキー_場コード_7']*7 + sed_df[ 'レースキー_場コード_8']*8 + sed_df[ 'レースキー_場コード_9']*9 + sed_df[ 'レースキー_場コード_10']*10 + sed_df['レースキー_R']*200



In [ ]:
res = sed_df["レースid"].value_counts(sort=True).where(lambda d:d>=2).dropna()
sed_df_not1 = sed_df[sed_df["レースid"].isin(res.index)]

In [ ]:
sed_df_not1['レースid'].value_counts()

202109112206    18
202110162404    18
202111201609    18
201412061207    18
202112181407    18
                ..
201402082410     2
201401182207     2
201401191806     2
201401191408     2
201401261007     2
Name: レースid, Length: 24093, dtype: int64

In [ ]:
X_columns = ['馬番',
       'レース条件_距離', 'レース場条件_重量',
       'レース場条件_頭数', '馬成績_斤量',
       '10時単勝オッズ', '10時複勝オッズ',
       '馬体重', '馬場係数',
       '前走1_nsp', '前走2_nsp', '前走3_nsp', '前走1_nsp_前3f', '前走2_nsp_前3f',
       '前走3_nsp_前3f', '前走1_nsp_後3f', '前走2_nsp_後3f', '前走3_nsp_後3f',
       'レースキー_場コード_1', 'レースキー_場コード_2', 'レースキー_場コード_3', 'レースキー_場コード_4',
       'レースキー_場コード_5', 'レースキー_場コード_6', 'レースキー_場コード_7', 'レースキー_場コード_8',
       'レースキー_場コード_9', 'レースキー_場コード_10', 'レース条件_トラック情報_芝ダ障害コード_1',
       'レース条件_トラック情報_芝ダ障害コード_2', 'レース場条件_トラック情報_右左_1', 'レース場条件_トラック情報_右左_2',
       'レース場条件_トラック情報_右左_3', 'レース場条件_トラック情報_内外_1', 'レース場条件_トラック情報_内外_2',
       'レース場条件_トラック情報_内外_9', 'レース場条件_馬場状態_10', 'レース場条件_馬場状態_11',
       'レース場条件_馬場状態_12', 'レース場条件_馬場状態_20', 'レース場条件_馬場状態_21', 'レース場条件_馬場状態_22',
       'レース場条件_馬場状態_30', 'レース場条件_馬場状態_31', 'レース場条件_馬場状態_32', 'レース場条件_馬場状態_40',
       'レース場条件_馬場状態_41', 'レース場条件_馬場状態_42', 'レース場条件_種別_11', 'レース場条件_種別_12',
       'レース場条件_種別_13', 'レース場条件_種別_14', 'レース場条件_条件_05', 'レース場条件_条件_10',
       'レース場条件_条件_16', 'レース場条件_条件_A3', 'レース場条件_条件_OP',]
y_columns = ['馬成績_着順_uma']

In [ ]:
len(X_columns)

57

In [ ]:
train = sed_df_not1[sed_df_not1['競争成績キー_年月日'] < 20200700]
valid = sed_df_not1[(sed_df_not1['競争成績キー_年月日'] < 20210000) & (sed_df_not1['競争成績キー_年月日'] > 20200700)]
test = sed_df_not1[sed_df_not1['競争成績キー_年月日'] > 20210000]
train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)
test = test.reset_index(drop=True)
print(len(train))
print(len(valid))
print(len(test))


268786
18895
40537


In [ ]:
d = train.groupby('レースid').groups
dd = pd.Series(d)
train_index_list =dd.reset_index(drop=True)

d = valid.groupby('レースid').groups
dd = pd.Series(d)
valid_index_list =dd.reset_index(drop=True)

d = test.groupby('レースid').groups
dd = pd.Series(d)
test_index_list =dd.reset_index(drop=True)

In [ ]:
train_X = train[X_columns]
train_y = train[y_columns]

valid_X = valid[X_columns]
valid_y = valid[y_columns]

test_X = test[X_columns]
test_y = test[y_columns]

### ジェネレータ

同一レースから2データをサンプリングするためにジェネレータクラスを定義した。

In [ ]:
class RankNetGenerator(object):

    def __init__(self, X, y, race_index_list, X_scaler=None, batch_size=32):
        self._X = X

        self._y = y
        # 同一レースのindexを保持するpandas.Seriesのリスト
        self._race_index_list = race_index_list

        # 説明変数に適用する標準化スケーラ
        self._X_scaler = X_scaler 

        # バッチサイズ
        self._batch_size = batch_size

    def __iter__(self):
        return self

    def __len__(self):
        return len(self._X)

    def __next__(self):

        # レースをサンプリング
        race_indexes = np.random.randint(0, len(self._race_index_list), self._batch_size)
        race_indexes = [self._race_index_list[race_index].values for race_index in race_indexes]

        # 出走馬をサンプリング
        uma_indexes = [race_index[np.random.choice(race_index.shape[0], 2, replace=False)] for race_index in race_indexes]
        # タプルの先頭要素がランクが高くなるようにする。ここについては要検討。yを1にする必要はないかも。
        uma_indexes = [(idx_1, idx_2) if self._y.loc[idx_1].values < self._y.loc[idx_2].values else (idx_2, idx_1) for idx_1, idx_2 in uma_indexes]

        # 説明変数の設定
        X = np.array([(self._X.loc[uma_index[0]], self._X.loc[uma_index[1]]) for uma_index in uma_indexes])
        X = [self._X_scaler.transform(X[:, 0, :]), self._X_scaler.transform(X[:, 1, :])]

        # サンプルした2データの先頭が必ずランクが大きくなるため、全て１.
        y = np.ones(self._batch_size)

        return X, y

In [ ]:
import itertools

class RankNetTestGeneratorForRace(object):

    def __init__(self, X, X_scaler=None):

        # 説明変数
        self._X = X

        # 説明変数に適用する標準化スケーラ
        self._X_scaler = X_scaler 

    def __iter__(self):
        return self

    def __len__(self):
        return len(self._X)

    def __next__(self):

        run_n = len(self._X)
        umas = []
        for b in range(run_n):
          umas.append(b)
        uma_indexes = list(itertools.permutations(umas, 2))
        batch_size = len(uma_indexes)
        

        # 説明変数の設定
        X = np.array([(self._X.loc[uma_index[0]], self._X.loc[uma_index[1]]) for uma_index in uma_indexes])
        X = [self._X_scaler.transform(X[:, 0, :]), self._X_scaler.transform(X[:, 1, :])]

        # 真の分布\bar{p_{ij}}の設定
        # サンプルした2データの先頭が必ずランクが大きくなるため、全て１.
        y = np.ones(batch_size)

        return X, y

### モデル

モデルはTensorflowによって作った。2つの入力を受け、モデルの出力差分、および活性化関数としてsigmoidを返す。

In [ ]:
import tensorflow as tf

def build_nn(input_dim, hidden_dim, output_dim, layer_num, activation, dropout_rate):
    inputs = tf.keras.Input(shape=(input_dim,))

    x = tf.keras.layers.Dense(hidden_dim, activation=activation)(inputs)
    for i in range(layer_num):
        x = tf.keras.layers.Dense(hidden_dim, activation=activation)(x)
        x = tf.keras.layers.Dropout(dropout_rate)(x)

    outputs = tf.keras.layers.Dense(output_dim)(x)

    return tf.keras.Model(inputs=inputs, outputs=outputs)

def build_ranknet(input_dim, hidden_dim, output_dim, layer_num, activation, dropout_rate):
    inputs_1 = tf.keras.Input(shape=(input_dim,))
    inputs_2 = tf.keras.Input(shape=(input_dim,))

    nn = build_nn(input_dim=input_dim, 
                  hidden_dim=hidden_dim,
                  output_dim=output_dim, 
                  layer_num=layer_num,
                  activation=activation,
                  dropout_rate=dropout_rate, 
    )

    x1 = nn(inputs_1)
    x2 = nn(inputs_2)

    subtract = tf.keras.layers.Subtract()([x1, x2])
    outputs = tf.keras.layers.Activation('sigmoid')(subtract)

    return tf.keras.Model(inputs=[inputs_1, inputs_2], outputs=outputs)


### 学習

Early Stoppingは、validationスコアが5spoch連続で進まなかった場合に行う。

In [ ]:
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import StandardScaler

# ハイパーパラメータ
input_dim = 57                 # モデルの入力(説明変数)の次元数
output_dim = 1                  # モデルの出力次元数
hidden_dim = 30                # 隠れ層の次元数
layer_num = 3                   # 隠れ層数
activation = tf.nn.relu         # 活性化関数
dropout_rate = 0.1              # ドロップアウト率
epoch = 100                     # エポック数
train_steps = 1000              # 学習データでの１エポック当たりのステップ数
valid_steps = 100               # 開発データでの１エポック当たりのステップ数
loss = "binary_crossentropy"    # 損失関数
optimizer = "adam"              # オプティマイザ

# 標準化
X_scaler = StandardScaler()
_ = X_scaler.fit_transform(train_X.values)

# ジェネレータ
train_generator = RankNetGenerator(train_X, train_y, train_index_list, X_scaler=X_scaler, batch_size=32)
valid_generator = RankNetGenerator(valid_X, valid_y, valid_index_list, X_scaler=X_scaler, batch_size=32)

# モデル
ranknet = build_ranknet(input_dim, hidden_dim, output_dim, layer_num, activation, dropout_rate)

# オプティマイザーと損失関数を設定
ranknet.compile(optimizer=optimizer,
                loss=loss)

# Early Stopping
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)]

# 学習
model_ranknet = ranknet.fit(train_generator, 
                            validation_data=valid_generator, 
                            epochs=epoch,
                            batch_size=32,
                            callbacks=callbacks, 
                            steps_per_epoch=train_steps,
                            validation_steps=valid_steps)


Epoch 1/100
1000/1000 [==============================] - 32s 30ms/step - loss: 0.6198 - val_loss: 0.5920
Epoch 2/100
1000/1000 [==============================] - 28s 28ms/step - loss: 0.5795 - val_loss: 0.5858
Epoch 3/100
1000/1000 [==============================] - 28s 28ms/step - loss: 0.5742 - val_loss: 0.5840
Epoch 4/100
1000/1000 [==============================] - 29s 29ms/step - loss: 0.5730 - val_loss: 0.5940
Epoch 5/100
1000/1000 [==============================] - 28s 28ms/step - loss: 0.5667 - val_loss: 0.5731
Epoch 6/100
1000/1000 [==============================] - 27s 27ms/step - loss: 0.5706 - val_loss: 0.5739
Epoch 7/100
1000/1000 [==============================] - 27s 27ms/step - loss: 0.5632 - val_loss: 0.5904
Epoch 8/100
1000/1000 [==============================] - 28s 28ms/step - loss: 0.5653 - val_loss: 0.5867
Epoch 9/100
1000/1000 [==============================] - 28s 28ms/step - loss: 0.5718 - val_loss: 0.5838
Epoch 10/100
1000/1000 [==============================]

## 予測

予測は以下のフローで行う。

- 馬Aが馬Bに勝利し、馬Bが馬Cに勝利した場合、馬Aは馬Cに勝利していることを期待し、3すくみの結果になることがない場合
1. 各レースに出走する全ての馬での総当たり戦を行う。すなわち、n頭出走する場合はnC2回モデルがpredictionを行う。
2. 総当たり戦結果を用い、なんか頑張って1列に順位通り並べる。

- そうはならない場合(多分こっち)
1. 各馬に対し、他の馬との対戦の勝率を合計する。
2. それが勝率。

In [ ]:
def predict(model, test_x, X_scaler):
  # ジェネレータ呼び出し
  generator = RankNetTestGeneratorForRace(test_x, X_scaler=X_scaler)
  # 予測
  ans = model.predict(generator, steps=1)
  """以下、ランキング付けを行う。現在は勝率の単純累計"""
  uma_n = len(test_x)
  syouritu = [0] * uma_n
  for index, n in enumerate(ans):
    uma_index =  (index // (uma_n -1 ))
    syouritu[uma_index] += n
  df = pd.DataFrame(syouritu, columns = ['syouritu'])
  df['pred_rank'] = df['syouritu'].rank(ascending=False)
  return df

In [ ]:
rank1_score = 0
ranksum_score = 0
count = 0
money_tansyo = 0
fukusyo_count = 0
money_fukusyo = 0

honki12_tansyo = 0
honki12_count = 0
honki12_count_hit = 0

honki11_tansyo = 0
honki11_count = 0
honki11_count_hit = 0


for index, idx in enumerate(test_index):
  if index%100==0:
    print(index)
  test2 = test[test['レースid']==int(idx)]
  test2 = test2.reset_index(drop=True)
  test2_X = test2[X_columns]
  test2_y = test2[y_columns]
  itii = 20
  ni = 20
  if len(test2) > 0:
    count += 1
    X_scaler = StandardScaler()
    _ = X_scaler.fit_transform(test2_X.values)
    # 予測
    pred = predict(ranknet, test2_X, X_scaler)
    for n in range(len(test2_X)):
      if int(pred['pred_rank'][n]) == 1:
        itii = n
      if int(pred['pred_rank'][n]) == 2:
        ni = n
      if (int(test2_y['馬成績_着順_uma'][n]) == 1) and (int(test2_y['馬成績_着順_uma'][n]) == int(pred['pred_rank'][n])):
        rank1_score += 1
        money_tansyo += test2_X['10時単勝オッズ'][n]
      if (int(test2_y['馬成績_着順_uma'][n]) < 4) and (int(pred['pred_rank'][n]) < 4):
        money_fukusyo += test2_X['10時複勝オッズ'][n]
      if (int(pred['pred_rank'][n]) < 4):
        fukusyo_count += 1
      ranksum_score += (int(test2_y['馬成績_着順_uma'][n]) - int(pred['pred_rank'][n]))**2

    if (itii < 20) and (ni < 20) and (pred['syouritu'][itii] > (pred['syouritu'][ni] * 1.2)):
      honki12_count += 1
      if int(test2_y['馬成績_着順_uma'][itii]) == 1:
        honki12_count_hit += 1
        honki12_tansyo += test2_X['10時単勝オッズ'][itii]

    if (itii < 20) and (ni < 20) and (pred['syouritu'][itii] > (pred['syouritu'][ni] * 1.1)):
      honki11_count += 1
      if int(test2_y['馬成績_着順_uma'][itii]) == 1:
        honki11_count_hit += 1
        honki11_tansyo += test2_X['10時単勝オッズ'][itii]
        
    
print(count)
print(rank1_score)
print(ranksum_score)
print(money_tansyo)
print(fukusyo_count)
print(money_fukusyo)

In [ ]:
print('サンプルレース数：{}件'.format(count))
print('単勝的中率：{}%'.format(100 * rank1_score/count))
print('全着順MSE：{}'.format(ranksum_score/len(test)))
print('単勝回収率：{}%'.format(100 * money_tansyo/count))
print('複勝回収率：{}%'.format(100 * money_fukusyo/fukusyo_count))

print('確率1.2倍時単勝購入回数：{}回'.format(honki12_count))
print('確率1.2倍時単勝的中回数：{}回'.format(honki12_count_hit))
print('確率1.2倍時単勝回収率：{}%'.format(100 * honki12_tansyo/honki12_count))

print('確率1.1倍時単勝購入回数：{}回'.format(honki11_count))
print('確率1.1倍時単勝的中回数：{}回'.format(honki11_count_hit))
print('確率1.1倍時単勝回収率：{}%'.format(100 * honki11_tansyo/honki11_count))


サンプルレース数：3031件
単勝的中率：30.419003629165292%
全着順MSE：16.754791918494217
単勝回収率：93.68195315077536%
複勝回収率：83.11888265698951%
確率1.2倍時単勝購入回数：97回
確率1.2倍時単勝的中回数：38回
確率1.2倍時単勝回収率：84.74226804123711%
確率1.1倍時単勝購入回数：580回
確率1.1倍時単勝的中回数：235回
確率1.1倍時単勝回収率：98.62068965517244%


ちなみに、全く同じことをもう一度した場合の成果は下のような感じ。

In [ ]:
print('サンプルレース数：{}件'.format(count))
print('単勝的中率：{}%'.format(100 * rank1_score/count))
print('全着順MSE：{}'.format(ranksum_score/len(test)))
print('単勝回収率：{}%'.format(100 * money_tansyo/count))
print('複勝回収率：{}%'.format(100 * money_fukusyo/fukusyo_count))

print('確率1.2倍時単勝購入回数：{}回'.format(honki12_count))
print('確率1.2倍時単勝的中回数：{}回'.format(honki12_count_hit))
print('確率1.2倍時単勝回収率：{}%'.format(100 * honki12_tansyo/honki12_count))

print('確率1.1倍時単勝購入回数：{}回'.format(honki11_count))
print('確率1.1倍時単勝的中回数：{}回'.format(honki11_count_hit))
print('確率1.1倍時単勝回収率：{}%'.format(100 * honki11_tansyo/honki11_count))


サンプルレース数：3031件
単勝的中率：31.111844275816562%
全着順MSE：16.76525149863088
単勝回収率：95.20950181458265%
複勝回収率：83.33553282745034%
確率1.2倍時単勝購入回数：90回
確率1.2倍時単勝的中回数：34回
確率1.2倍時単勝回収率：79.66666666666667%
確率1.1倍時単勝購入回数：549回
確率1.1倍時単勝的中回数：204回
確率1.1倍時単勝回収率：90.78324225865212%


## 結論

- 前走データ、馬場状態データのみから回収率約94%を達成

## 考察

- 以前、lgbmの2値分類を用いて同じ特徴量で学習を行い、全てのレースで馬券を購入した際、回収率は70%であった。今回は回収率が94%であったため、目覚ましい成果。
- 今回勝率を計算する際、めちゃくちゃ単純な計算を行った。改善の余地がありそう。
- 何度か全く同じ学習を行った際、単勝回収率は92-95%くらいのブレがあった。
- 使用している特徴量がまだまだ全然足りない。増やしたらどの程度精度が変わるだろうか。

## 議論

### 特徴量に関して
- 騎手データや調教データを入れたい。騎手データはどんな形にすればいいだろうか。
  - 勝率、連対率、3着内率から1位、2位、3位の確率を算出して使う

### 学習に関して
- 今回は学習が進んだが、データ量を半年ほど増やすと損失関数が発散してしまった(`loss:nan`になった)。対処方法がいまいちよくわからない。
  - 別のcsvからデータを統合した際の型のエラーだった。解決済み。

### 予測に関して
- 勝率の計算方法が分からない。1vs1の成績データがいっぱいあるのだから、絶対もっといいやり方があるはず。
- ものの数回の学習でも3%程度の精度にずれが出た。また、確率を絞った買い方をした際に(サンプルが少ないこともあって)「より絞った買い方をした方が精度が下がる」なんてこともあった。具体的にどの程度のサンプル数を持てば十分な精度検証ができるのだろうか？
  - 5000件での精度を正解とする
- 馬券の買い方について。勝率を出せたら単勝は十分な買い方になりうるが、馬単や三連単の方が適してるかも？

## 準備
- 当たっているデータ、当たっていないデータの具体例をピックアップする
- 人間のベンチマーク
  - 一番人気だけを買い続ける、的な
  